In [61]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime

# from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score #kfold
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

- https://www.youtube.com/watch?v=irHhDMbw3xo
- logistic regression's solver : https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-definitions

In [2]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [3]:
df.shape

(891, 12)

In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
df.Embarked.notna()

0      True
1      True
2      True
3      True
4      True
       ... 
886    True
887    True
888    True
889    True
890    True
Name: Embarked, Length: 891, dtype: bool

In [10]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]

In [11]:
df.shape

(889, 4)

In [13]:
df.head()
# survived is the target

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [15]:
X = df.loc[:, ['Pclass']]
y = df.Survived

In [16]:
X.shape

(889, 1)

In [17]:
y.shape

(889,)

In [22]:
logreg = LogisticRegression(solver='lbfgs')

In [27]:
print(cross_val_score(logreg, X, y, cv=5)) # kfold cross validation
cross_val_score(logreg, X, y, cv=5).mean()

[0.6011236  0.6741573  0.6741573  0.71910112 0.72316384]


0.6783406335301212

In [28]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [29]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


- ohe
- sparse=False returns array, sparse=True returns sparse matrix
- sparse=True can save memories
- when use sparse=True, you shoud do ohe.fit_transform(..).to_array() << something like this method to change matrix to array 

In [38]:
ohe = OneHotEncoder(sparse=False) # 

In [46]:
ohe.fit_transform(df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [47]:
ohe.categories_ 
# first column represents female, second does for male

[array(['female', 'male'], dtype=object)]

In [48]:
ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [50]:
ohe.categories_ 

[array(['C', 'Q', 'S'], dtype=object)]

###  use pipeline to perfrom the exact process done before

In [51]:
X = df.drop('Survived', axis='columns')

In [52]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


- Use make_column_transformer method when we need different types of preprocessing
- e.g. we have to do ohe for 'Sex' and 'Embarked'  but not for 'Pclass', then make_column_transformer just do ohe for those two
- remainder='passthrough' leaves the rest columns as they were which haven't preprocessed

In [55]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough') 

In [60]:
column_trans.fit_transform(X)
# first two columns are ohe 'sex', second three columns are ohe 'Embakred', the last one is 'pclass'

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

#####  Pipeline step

In [62]:
pipe = make_pipeline(column_trans, logreg)

In [64]:
print(cross_val_score(pipe, X, y, cv=5, scoring='accuracy'))
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

[0.76404494 0.79213483 0.76966292 0.75280899 0.78531073]


0.7727924839713071

In [66]:
X_new = X.sample(5, random_state=99)
X_new


,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


- If you use get_dummies() from pandas, you have to do get_dummies() for everysingle time you put new data on it

In [67]:
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [69]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)